# Demo Minggu 3: Analisis Frekuensi
## *Frequency Analysis of Biomedical Signals*

**Mata Kuliah:** Pengolahan Sinyal Medis -- Universitas Indonesia

---

### Tujuan Pembelajaran

Setelah mengikuti demo ini, mahasiswa diharapkan dapat:

1. Memahami keterbatasan analisis domain waktu dan motivasi analisis frekuensi (*frequency domain analysis*).
2. Menghitung dan menginterpretasikan **Discrete Fourier Transform (DFT)** menggunakan algoritma **FFT** (*Fast Fourier Transform*).
3. Memahami konsep **resolusi frekuensi** dan **frekuensi Nyquist** dalam konteks FFT.
4. Menghitung **Power Spectral Density (PSD)** menggunakan metode Periodogram dan Welch.
5. Memvisualisasikan evolusi spektrum sinyal terhadap waktu menggunakan **Spectrogram** (*Short-Time Fourier Transform*, STFT).
6. Mengidentifikasi komponen frekuensi klinisnya penting pada sinyal ECG dan EEG.

---
## 1. Motivasi: Keterbatasan Domain Waktu

Pada minggu-minggu sebelumnya kita menganalisis sinyal di **domain waktu** (*time domain*) -- memplot amplitudo sinyal sebagai fungsi waktu. Pendekatan ini memiliki keterbatasan:

- **Noise dan sinyal sering bercampur** di domain waktu sehingga sulit dibedakan secara visual.
- **Komponen frekuensi** yang menyusun sinyal tidak terlihat langsung.
- **Karakteristik ritmikal** (seperti irama jantung atau osilasi otak) lebih mudah dianalisis di domain frekuensi.

### Contoh Klinis: Komponen Frekuensi Sinyal Biomedis

| Sinyal | Komponen | Rentang Frekuensi | Makna Klinis |
|--------|----------|-------------------|--------------|
| **ECG** | *Baseline wander* | 0 -- 0.5 Hz | Artefak pernapasan |
| **ECG** | Gelombang P | 0.5 -- 2 Hz | Depolarisasi atrium |
| **ECG** | Gelombang T | 1 -- 5 Hz | Repolarisasi ventrikel |
| **ECG** | Kompleks QRS | 5 -- 40 Hz | Depolarisasi ventrikel |
| **ECG** | *Power line noise* | 50 Hz | Interferensi jala listrik |
| **EEG** | Delta ($\delta$) | 0.5 -- 4 Hz | Tidur nyenyak |
| **EEG** | Theta ($\theta$) | 4 -- 8 Hz | Mengantuk, meditasi |
| **EEG** | Alpha ($\alpha$) | 8 -- 13 Hz | Relaksasi, mata tertutup |
| **EEG** | Beta ($\beta$) | 13 -- 30 Hz | Konsentrasi, aktif |
| **EEG** | Gamma ($\gamma$) | 30 -- 50 Hz | Pemrosesan kognitif tinggi |

Dengan **analisis frekuensi**, kita dapat memisahkan dan mengidentifikasi komponen-komponen di atas secara kuantitatif.

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import signal as scipy_signal

# Library kursus
from medsinyal.io import load_synthetic
from medsinyal.viz import plot_signal, plot_signals
from medsinyal.filters import bandpass_filter, notch_filter, moving_average

# Konfigurasi plot global
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.grid.alpha'] = 0.4
plt.rcParams['lines.linewidth'] = 1.2

print('Semua library berhasil dimuat.')
print(f'NumPy  version: {np.__version__}')
print(f'SciPy  version: {scipy_signal.__version__}')

---
## 2. Discrete Fourier Transform (DFT) dan FFT

### 2.1 Teori DFT

**Discrete Fourier Transform (DFT)** mengubah sinyal diskret domain waktu $x[n]$ (panjang $N$ sampel) ke representasi domain frekuensi $X[k]$:

$$X[k] = \sum_{n=0}^{N-1} x[n] \, e^{-j 2\pi kn/N}, \quad k = 0, 1, \ldots, N-1$$

Dengan invers DFT:

$$x[n] = \frac{1}{N} \sum_{k=0}^{N-1} X[k] \, e^{j 2\pi kn/N}$$

**Spektrum magnitudo** (*magnitude spectrum*) diperoleh dengan:

$$|X[k]| = \sqrt{\text{Re}(X[k])^2 + \text{Im}(X[k])^2}$$

### 2.2 Fast Fourier Transform (FFT)

**FFT** adalah algoritma efisien untuk menghitung DFT. Kompleksitas komputasi:

| Algoritma | Kompleksitas |
|-----------|-------------|
| DFT naif | $O(N^2)$ |
| **FFT** (Cooley-Tukey) | $O(N \log N)$ |

Untuk $N = 10^6$: DFT butuh $10^{12}$ operasi, FFT hanya $\approx 2 \times 10^7$ -- **50.000x lebih cepat!**

### 2.3 Parameter Penting FFT

| Parameter | Rumus | Keterangan |
|-----------|-------|------------|
| **Resolusi frekuensi** | $\Delta f = f_s / N$ | Jarak antar bin frekuensi |
| **Frekuensi maksimum** | $f_{\max} = f_s / 2$ | Frekuensi Nyquist |
| **Durasi minimum** | $T_{\min} = 1 / \Delta f$ | Durasi sinyal untuk mencapai $\Delta f$ |

In [ ]:
# Muat data komposisi gelombang sinus
data_sine = load_synthetic('sine_compositions')

t    = data_sine['t']
fs   = float(data_sine['fs'])
N    = len(t)

print('=== Parameter Sinyal ===')
print(f'Sampling rate  (fs) : {fs:.0f} Hz')
print(f'Jumlah sampel  (N)  : {N}')
print(f'Durasi              : {t[-1]:.2f} s')
print()
print('=== Parameter FFT ===')
delta_f = fs / N
f_max   = fs / 2
print(f'Resolusi frekuensi  : Δf = fs/N = {fs:.0f}/{N} = {delta_f:.4f} Hz')
print(f'Frekuensi Nyquist   : f_max = fs/2 = {f_max:.0f} Hz')
print(f'Jumlah bin frekuensi: {N // 2 + 1} (one-sided)')

In [ ]:
# Demonstrasi FFT pada sinyal sinus sederhana (5 Hz)
simple_sine = data_sine['simple_sine']

# Hitung FFT
X_full   = np.fft.fft(simple_sine)           # FFT dua sisi (two-sided)
freqs    = np.fft.fftfreq(N, d=1.0 / fs)     # sumbu frekuensi two-sided
mag_full = np.abs(X_full) / N                 # normalisasi amplitudo

# Spektrum satu sisi (one-sided): ambil bin 0..N//2
n_pos    = N // 2 + 1
freqs_pos = freqs[:n_pos]
mag_pos   = mag_full[:n_pos] * 2              # kalikan 2 (energi dilipat)
mag_pos[0] /= 2                               # DC tidak dikalikan 2

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
fig.suptitle('FFT Gelombang Sinus 5 Hz', fontsize=13, fontweight='bold')

# Spektrum dua sisi
axes[0].plot(freqs, mag_full, 'b-', linewidth=0.9)
axes[0].set_title('Spektrum Dua Sisi (Two-Sided Spectrum)')
axes[0].set_xlabel('Frekuensi (Hz)')
axes[0].set_ylabel('Magnitudo')
axes[0].set_xlim([-fs/2, fs/2])

# Spektrum satu sisi
axes[1].plot(freqs_pos, mag_pos, 'r-', linewidth=0.9)
axes[1].set_title('Spektrum Satu Sisi (One-Sided Spectrum)')
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('Magnitudo')
axes[1].set_xlim([0, fs/2])

plt.tight_layout()
plt.show()

# Tampilkan puncak frekuensi
peak_idx = np.argmax(mag_pos[1:]) + 1   # abaikan DC
print(f'Puncak frekuensi terdeteksi: {freqs_pos[peak_idx]:.2f} Hz (diharapkan: 5 Hz)')
print(f'Amplitudo pada puncak      : {mag_pos[peak_idx]:.4f} (diharapkan: ~1.0)')
print()
print('Catatan: Spektrum dua sisi mengandung komponen frekuensi positif dan negatif.')
print('Spektrum satu sisi lebih intuitif -- hanya frekuensi positif yang ditampilkan.')

In [ ]:
# FFT pada sinyal multi-frekuensi (1 + 5 + 12 + 30 Hz)
multi_freq = data_sine['multi_freq']

X_multi   = np.fft.fft(multi_freq)
mag_multi = np.abs(X_multi) / N
mag_multi_pos = mag_multi[:n_pos] * 2
mag_multi_pos[0] /= 2

fig, axes = plt.subplots(2, 1, figsize=(13, 7), sharex=False)
fig.suptitle('FFT Sinyal Multi-Frekuensi (1 + 5 + 12 + 30 Hz)', fontsize=13, fontweight='bold')

# Domain waktu
axes[0].plot(t, multi_freq, 'k-', linewidth=0.8)
axes[0].set_xlabel('Waktu (s)')
axes[0].set_ylabel('Amplitudo')
axes[0].set_title('Domain Waktu -- Sinyal Gabungan')
axes[0].set_xlim([0, t[-1]])

# Spektrum frekuensi (one-sided)
axes[1].plot(freqs_pos, mag_multi_pos, 'tab:blue', linewidth=0.9)
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('Magnitudo')
axes[1].set_title('Domain Frekuensi -- Spektrum Magnitudo (One-Sided)')
axes[1].set_xlim([0, 60])

# Tandai puncak komponen
expected_freqs = [1, 5, 12, 30]
for f_comp in expected_freqs:
    idx_comp = np.argmin(np.abs(freqs_pos - f_comp))
    axes[1].axvline(x=freqs_pos[idx_comp], color='red', linestyle='--', alpha=0.6,
                    linewidth=1.2, label=f'{f_comp} Hz' if f_comp == 1 else None)
    axes[1].annotate(f'{f_comp} Hz',
                     xy=(freqs_pos[idx_comp], mag_multi_pos[idx_comp]),
                     xytext=(freqs_pos[idx_comp] + 1.5, mag_multi_pos[idx_comp] + 0.02),
                     fontsize=10, color='red',
                     arrowprops=dict(arrowstyle='->', color='red', lw=1.2))

plt.tight_layout()
plt.show()

print('Setiap puncak (spike) dalam spektrum sesuai dengan satu komponen sinusoidal.')
print('FFT berhasil memisahkan empat komponen yang tercampur di domain waktu!')
for f_comp in expected_freqs:
    idx_comp = np.argmin(np.abs(freqs_pos - f_comp))
    print(f'  {f_comp:2d} Hz --> magnitudo terdeteksi: {mag_multi_pos[idx_comp]:.3f}')

---
## 3. Interpretasi Spektrum Sinyal Biomedis

### 3.1 Spektrum ECG: Bersih vs. Terkontaminasi Noise

Pada sinyal ECG terkontaminasi, komponen noise akan terlihat jelas di domain frekuensi:

- **Pita rendah (0--2 Hz)**: *Baseline wander* akibat pernapasan
- **Pita tengah (5--40 Hz)**: Komponen diagnostik ECG (QRS, P, T)
- **Puncak 50 Hz**: Interferensi jala listrik (*power line interference*)

Dengan melihat spektrum, kita dapat:
1. Mengidentifikasi jenis noise yang ada
2. Merancang filter yang tepat untuk menghilangkannya
3. Memverifikasi efektivitas filter setelah diterapkan

In [ ]:
# Muat data ECG
data_ecg = load_synthetic('synthetic_ecg')

t_ecg      = data_ecg['t']
fs_ecg     = float(data_ecg['fs'])
ecg_clean  = data_ecg['ecg_clean']
ecg_noisy  = data_ecg['ecg_noisy']
N_ecg      = len(t_ecg)

print(f'Sampling rate ECG : {fs_ecg:.0f} Hz')
print(f'Jumlah sampel     : {N_ecg}')
print(f'Durasi sinyal     : {t_ecg[-1]:.1f} s')
print(f'Resolusi frekuensi: {fs_ecg/N_ecg:.4f} Hz')

In [ ]:
def compute_spectrum(x, fs):
    """Hitung spektrum magnitudo satu sisi yang dinormalisasi.

    Parameters
    ----------
    x : np.ndarray
        Sinyal input.
    fs : float
        Sampling rate (Hz).

    Returns
    -------
    freqs : np.ndarray
        Vektor frekuensi (Hz), 0 hingga fs/2.
    mag : np.ndarray
        Magnitudo spektrum satu sisi.
    """
    N   = len(x)
    X   = np.fft.fft(x)
    mag = np.abs(X) / N
    # Ambil sisi positif
    n_pos = N // 2 + 1
    freqs = np.fft.fftfreq(N, d=1.0 / fs)[:n_pos]
    mag   = mag[:n_pos] * 2
    mag[0] /= 2  # DC tidak dilipatgandakan
    return freqs, mag


# Hitung spektrum ECG bersih dan noisy
freqs_ecg, mag_clean = compute_spectrum(ecg_clean, fs_ecg)
_,          mag_noisy = compute_spectrum(ecg_noisy, fs_ecg)

fig, axes = plt.subplots(3, 1, figsize=(13, 11))
fig.suptitle('Perbandingan Spektrum ECG: Bersih vs. Terkontaminasi Noise', fontsize=13, fontweight='bold')

# Sinyal domain waktu
axes[0].plot(t_ecg, ecg_clean, 'tab:blue', linewidth=0.8, label='ECG Bersih')
axes[0].plot(t_ecg, ecg_noisy, 'tab:red', linewidth=0.6, alpha=0.7, label='ECG Noisy')
axes[0].set_title('Domain Waktu')
axes[0].set_xlabel('Waktu (s)')
axes[0].set_ylabel('Amplitudo (mV)')
axes[0].legend(loc='upper right')

# Spektrum ECG bersih
axes[1].plot(freqs_ecg, mag_clean, 'tab:blue', linewidth=0.9)
axes[1].set_title('Spektrum Magnitudo -- ECG Bersih')
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('Magnitudo (mV)')
axes[1].set_xlim([0, fs_ecg/2])
# Anotasi pita frekuensi ECG
axes[1].axvspan(0,    0.5,  alpha=0.10, color='purple', label='Baseline wander')
axes[1].axvspan(5,    40,   alpha=0.10, color='green',  label='QRS (5-40 Hz)')

# Spektrum ECG noisy
axes[2].plot(freqs_ecg, mag_noisy, 'tab:red', linewidth=0.9)
axes[2].set_title('Spektrum Magnitudo -- ECG Terkontaminasi Noise')
axes[2].set_xlabel('Frekuensi (Hz)')
axes[2].set_ylabel('Magnitudo (mV)')
axes[2].set_xlim([0, fs_ecg/2])
axes[2].axvspan(0,    0.5,  alpha=0.10, color='purple', label='Baseline wander (0-0.5 Hz)')
axes[2].axvspan(5,    40,   alpha=0.10, color='green',  label='QRS (5-40 Hz)')
# Tandai komponen 50 Hz
idx_50 = np.argmin(np.abs(freqs_ecg - 50))
axes[2].axvline(x=50, color='darkorange', linestyle='--', linewidth=1.5, label='50 Hz (power line)')
axes[2].annotate('50 Hz\n(interferensi\njala listrik)',
                 xy=(50, mag_noisy[idx_50]),
                 xytext=(55, mag_noisy[idx_50] * 0.8),
                 fontsize=9, color='darkorange',
                 arrowprops=dict(arrowstyle='->', color='darkorange', lw=1.2))

for ax in axes[1:]:
    ax.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

print(f'Puncak pada 50 Hz (noisy): magnitudo = {mag_noisy[idx_50]:.4f} mV')
print(f'Puncak pada 50 Hz (clean): magnitudo = {mag_clean[idx_50]:.4f} mV')
print()
print('Puncak 50 Hz terlihat jelas di spektrum ECG noisy -- ini adalah tanda khas')
print('interferensi jala listrik yang perlu dihilangkan dengan notch filter.')

### 3.2 Spektrum EEG: Pita Frekuensi Otak

EEG merupakan superposisi dari beberapa osilasi dengan frekuensi yang berbeda. Spektrum EEG memperlihatkan **distribusi daya** di setiap pita frekuensi:

$$\delta < \theta < \alpha < \beta < \gamma$$

Dalam data sintetis kita terdapat **alpha burst** (lonjakan aktivitas 8--13 Hz) pada $t = 10$--$15$ s yang mensimulasikan kondisi mata tertutup.

In [ ]:
# Muat data EEG
data_eeg = load_synthetic('synthetic_eeg')

t_eeg     = data_eeg['t']
fs_eeg    = float(data_eeg['fs'])
eeg_clean = data_eeg['eeg_clean']
N_eeg     = len(t_eeg)

print(f'Sampling rate EEG : {fs_eeg:.0f} Hz')
print(f'Jumlah sampel     : {N_eeg}')
print(f'Durasi sinyal     : {t_eeg[-1]:.1f} s')
print(f'Resolusi frekuensi: {fs_eeg/N_eeg:.4f} Hz')

# Hitung spektrum EEG
freqs_eeg, mag_eeg = compute_spectrum(eeg_clean, fs_eeg)

fig, axes = plt.subplots(2, 1, figsize=(13, 8))
fig.suptitle('Sinyal EEG dan Spektrum Frekuensinya', fontsize=13, fontweight='bold')

# Domain waktu
axes[0].plot(t_eeg, eeg_clean, 'k-', linewidth=0.7)
axes[0].set_title('Domain Waktu -- EEG Bersih')
axes[0].set_xlabel('Waktu (s)')
axes[0].set_ylabel('Amplitudo ($\\mu$V)')
# Tandai alpha burst
axes[0].axvspan(10, 15, alpha=0.15, color='gold', label='Alpha burst (10-15 s)')
axes[0].legend(loc='upper right')

# Spektrum frekuensi dengan arsiran pita
axes[1].plot(freqs_eeg, mag_eeg, 'k-', linewidth=0.9)
axes[1].set_title('Spektrum Magnitudo -- EEG (Semua Pita Frekuensi)')
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('Magnitudo ($\\mu$V)')
axes[1].set_xlim([0, 60])

# Arsir pita EEG
eeg_bands_def = [
    ('Delta',  0.5,  4,   'tab:blue'),
    ('Theta',  4,    8,   'tab:orange'),
    ('Alpha',  8,    13,  'tab:green'),
    ('Beta',   13,   30,  'tab:red'),
    ('Gamma',  30,   50,  'tab:purple'),
]
for band_name, f_lo, f_hi, color in eeg_bands_def:
    axes[1].axvspan(f_lo, f_hi, alpha=0.18, color=color, label=f'{band_name} ({f_lo}-{f_hi} Hz)')

axes[1].legend(loc='upper right', fontsize=8, ncol=2)
plt.tight_layout()
plt.show()

# Hitung magnitudo rata-rata di setiap pita
print('Magnitudo rata-rata per pita EEG:')
for band_name, f_lo, f_hi, _ in eeg_bands_def:
    mask = (freqs_eeg >= f_lo) & (freqs_eeg <= f_hi)
    mean_mag = np.mean(mag_eeg[mask])
    print(f'  {band_name:<6} ({f_lo:4.1f}-{f_hi:4.1f} Hz): {mean_mag:.4f} µV')

---
## 4. Power Spectral Density (PSD)

### 4.1 Apa itu PSD?

**Power Spectral Density (PSD)** menggambarkan bagaimana **daya** (*power*) sinyal terdistribusi di setiap frekuensi. Satuan PSD adalah $\text{V}^2/\text{Hz}$ (atau $\mu\text{V}^2/\text{Hz}$ untuk EEG).

$$P_{xx}(f) = \lim_{T \to \infty} \frac{1}{T} \left| X_T(f) \right|^2$$

PSD berguna karena:
- **Invariant terhadap panjang sinyal** (dinormalisasi per Hz)
- **Lebih stabil** secara statistik dibanding spektrum magnitudo
- **Langsung terbaca** sebagai "berapa banyak daya di pita frekuensi ini?"

### 4.2 Metode Estimasi PSD

| Metode | Cara Kerja | Kelebihan | Kekurangan |
|--------|-----------|-----------|------------|
| **Periodogram** | $P_{xx}[k] = \frac{1}{N \cdot f_s} |X[k]|^2$ | Sederhana, resolusi frekuensi tinggi | Variansi tinggi (noise) |
| **Welch** | Rata-rata periodogram dari segmen overlap | Variansi rendah, lebih halus | Resolusi frekuensi lebih rendah |

In [ ]:
# Hitung PSD dengan dua metode: Periodogram dan Welch
# --- ECG ---
f_pgram_ecg, psd_pgram_ecg = scipy_signal.periodogram(ecg_clean, fs=fs_ecg)
f_welch_ecg, psd_welch_ecg = scipy_signal.welch(
    ecg_clean, fs=fs_ecg,
    nperseg=int(fs_ecg * 4),   # window 4 detik
    noverlap=int(fs_ecg * 2),  # overlap 50%
    window='hann'
)

print('=== PSD ECG ===')
print(f'Periodogram : {len(f_pgram_ecg)} bin, resolusi {f_pgram_ecg[1]:.4f} Hz')
print(f'Welch       : {len(f_welch_ecg)} bin, resolusi {f_welch_ecg[1]:.4f} Hz')

fig, axes = plt.subplots(2, 1, figsize=(13, 8))
fig.suptitle('PSD Sinyal ECG: Periodogram vs. Metode Welch', fontsize=13, fontweight='bold')

# Plot linear
axes[0].plot(f_pgram_ecg, psd_pgram_ecg, 'tab:red',  linewidth=0.6, alpha=0.8, label='Periodogram')
axes[0].plot(f_welch_ecg, psd_welch_ecg, 'tab:blue', linewidth=1.4, label='Welch (4 s window, 50% overlap)')
axes[0].set_title('PSD ECG (Skala Linear)')
axes[0].set_xlabel('Frekuensi (Hz)')
axes[0].set_ylabel('PSD (mV²/Hz)')
axes[0].set_xlim([0, 80])
axes[0].legend(loc='upper right')

# Plot skala logaritmik (dB) -- lebih umum digunakan secara klinis
psd_pgram_db = 10 * np.log10(psd_pgram_ecg + 1e-15)
psd_welch_db = 10 * np.log10(psd_welch_ecg + 1e-15)
axes[1].plot(f_pgram_ecg, psd_pgram_db, 'tab:red',  linewidth=0.6, alpha=0.8, label='Periodogram')
axes[1].plot(f_welch_ecg, psd_welch_db, 'tab:blue', linewidth=1.4, label='Welch')
axes[1].set_title('PSD ECG (Skala Logaritmik dB)')
axes[1].set_xlabel('Frekuensi (Hz)')
axes[1].set_ylabel('PSD (dB re mV²/Hz)')
axes[1].set_xlim([0, 80])
axes[1].legend(loc='upper right')

plt.tight_layout()
plt.show()

print('Pengamatan:')
print('- Periodogram sangat "berisik" (variansi tinggi) -- kurva tidak mulus.')
print('- Welch menghasilkan estimasi yang lebih halus dengan mengorbankan sedikit resolusi.')
print('- Skala dB memudahkan membandingkan komponen dengan daya yang sangat berbeda.')

In [ ]:
# PSD EEG dengan metode Welch + arsiran pita frekuensi
f_welch_eeg, psd_welch_eeg = scipy_signal.welch(
    eeg_clean, fs=fs_eeg,
    nperseg=int(fs_eeg * 4),   # window 4 detik
    noverlap=int(fs_eeg * 2),  # overlap 50%
    window='hann'
)

fig, ax = plt.subplots(figsize=(13, 5))
ax.semilogy(f_welch_eeg, psd_welch_eeg, 'k-', linewidth=1.2, zorder=5)
ax.set_title('PSD Sinyal EEG (Welch) dengan Pita Frekuensi', fontsize=13, fontweight='bold')
ax.set_xlabel('Frekuensi (Hz)')
ax.set_ylabel('PSD ($\\mu$V²/Hz, skala log)')
ax.set_xlim([0.5, 55])

# Arsir pita dan hitung daya per pita
print('Daya relatif per pita EEG (Welch PSD):')
print(f'{"Pita":<8} {"Rentang":>12} {"Daya Total (µV²)":>20} {"Persentase":>12}')
print('-' * 56)

total_power = np.trapz(psd_welch_eeg[
    (f_welch_eeg >= 0.5) & (f_welch_eeg <= 50)
], f_welch_eeg[(f_welch_eeg >= 0.5) & (f_welch_eeg <= 50)])

for band_name, f_lo, f_hi, color in eeg_bands_def:
    mask = (f_welch_eeg >= f_lo) & (f_welch_eeg <= f_hi)
    band_power = np.trapz(psd_welch_eeg[mask], f_welch_eeg[mask])
    pct = 100.0 * band_power / total_power
    ax.fill_between(f_welch_eeg, psd_welch_eeg, where=mask,
                    alpha=0.35, color=color, label=f'{band_name} ({pct:.1f}%)')
    print(f'{band_name:<8} {f"{f_lo}-{f_hi} Hz":>12} {band_power:>20.4f} {pct:>11.1f}%')

ax.legend(loc='upper right', fontsize=9)
plt.tight_layout()
plt.show()

print(f'\nTotal daya (0.5-50 Hz): {total_power:.4f} µV²')
print('Pita alpha memiliki persentase tinggi karena adanya alpha burst (10-15 s).')

### 4.3 Pengaruh Parameter Metode Welch

Kualitas estimasi PSD Welch dipengaruhi oleh:

- **Panjang window** (*nperseg*): Lebih panjang → resolusi frekuensi lebih baik, variansi lebih tinggi
- **Overlap**: Lebih besar → lebih banyak rata-rata, variansi lebih rendah
- **Fungsi window**: Hann, Hamming, Blackman -- mempengaruhi *spectral leakage*

**Trade-off fundamental**: Resolusi frekuensi $\times$ Resolusi temporal = konstan (prinsip ketidakpastian).

In [ ]:
# Demonstrasi pengaruh panjang window Welch pada PSD EEG
window_durations = [
    (1,  'Window 1 s (resolusi rendah, variansi rendah)'),
    (4,  'Window 4 s (keseimbangan)'),
    (10, 'Window 10 s (resolusi tinggi, variansi tinggi)'),
]

fig, ax = plt.subplots(figsize=(13, 5))
ax.set_title('Pengaruh Panjang Window pada Estimasi PSD Welch (EEG)', fontsize=12, fontweight='bold')
ax.set_xlabel('Frekuensi (Hz)')
ax.set_ylabel('PSD ($\\mu$V²/Hz)')
ax.set_xlim([0, 55])

colors_win = ['tab:red', 'tab:blue', 'tab:green']
for (win_sec, label), color in zip(window_durations, colors_win):
    f_w, p_w = scipy_signal.welch(
        eeg_clean, fs=fs_eeg,
        nperseg=int(fs_eeg * win_sec),
        noverlap=int(fs_eeg * win_sec * 0.5),
        window='hann'
    )
    delta_f_w = f_w[1] - f_w[0]
    ax.plot(f_w, p_w, color=color, linewidth=1.1,
            label=f'{label} | Δf={delta_f_w:.3f} Hz')

ax.legend(loc='upper right', fontsize=8)
plt.tight_layout()
plt.show()

print('Pengamatan:')
print('- Window 1 s  : Kurva PSD halus tapi pita alpha melebar (resolusi rendah).')
print('- Window 10 s : Kurva lebih berfluktuasi (variansi tinggi) tapi pita lebih tajam.')
print('- Window 4 s  : Kompromi yang umum digunakan untuk analisis EEG klinis.')

---
## 5. Spectrogram (Short-Time Fourier Transform / STFT)

### 5.1 Motivasi: Sinyal Non-Stasioner

FFT global memberikan satu spektrum untuk **seluruh** sinyal. Namun, sinyal biomedis bersifat **non-stasioner** -- kandungan frekuensinya berubah seiring waktu. Misalnya:
- EEG: Transisi dari kondisi terjaga ke tidur
- ECG: Perubahan irama jantung (*aritmia*)

### 5.2 Short-Time Fourier Transform (STFT)

STFT menghitung FFT pada **jendela pendek yang bergeser** sepanjang sinyal:

$$\text{STFT}\{x[n]\}(m, k) = \sum_{n=-\infty}^{\infty} x[n] \, w[n - m] \, e^{-j2\pi kn/N}$$

di mana $w[n]$ adalah fungsi window dan $m$ adalah posisi jendela.

Hasilnya adalah **spectrogram** -- representasi dua dimensi (waktu $\times$ frekuensi) yang memperlihatkan bagaimana spektrum berevolusi dari waktu ke waktu.

### 5.3 Trade-off Resolusi

| Parameter | Window Pendek | Window Panjang |
|-----------|--------------|---------------|
| Resolusi waktu | Tinggi | Rendah |
| Resolusi frekuensi | Rendah | Tinggi |

Ini adalah manifestasi dari **Prinsip Ketidakpastian Heisenberg** dalam pemrosesan sinyal:

$$\Delta t \cdot \Delta f \geq \frac{1}{4\pi}$$

In [ ]:
# Spectrogram sinyal EEG -- memperlihatkan alpha burst pada t=10-15 s
nperseg_eeg = int(fs_eeg * 2)    # window 2 detik
noverlap_eeg = int(nperseg_eeg * 0.9)  # overlap 90%

f_spec_eeg, t_spec_eeg, Sxx_eeg = scipy_signal.spectrogram(
    eeg_clean, fs=fs_eeg,
    nperseg=nperseg_eeg,
    noverlap=noverlap_eeg,
    window='hann',
    scaling='density'
)

fig, axes = plt.subplots(2, 1, figsize=(14, 9))
fig.suptitle('Spectrogram Sinyal EEG', fontsize=13, fontweight='bold')

# Sinyal domain waktu
axes[0].plot(t_eeg, eeg_clean, 'k-', linewidth=0.7)
axes[0].axvspan(10, 15, alpha=0.2, color='gold', label='Alpha burst (10-15 s)')
axes[0].set_ylabel('Amplitudo ($\\mu$V)')
axes[0].set_title('EEG Domain Waktu')
axes[0].legend(loc='upper right')
axes[0].set_xlim([0, t_eeg[-1]])

# Spectrogram
im = axes[1].pcolormesh(
    t_spec_eeg, f_spec_eeg,
    10 * np.log10(Sxx_eeg + 1e-15),
    shading='gouraud', cmap='inferno'
)
axes[1].set_ylabel('Frekuensi (Hz)')
axes[1].set_xlabel('Waktu (s)')
axes[1].set_title(
    f'Spectrogram EEG (window={nperseg_eeg/fs_eeg:.0f} s, overlap={100*noverlap_eeg/nperseg_eeg:.0f}%)',
    fontsize=11
)
axes[1].set_ylim([0, 55])
# Garis pita EEG
for _, f_lo, f_hi, color in eeg_bands_def:
    axes[1].axhline(y=f_lo, color='white', linestyle='--', linewidth=0.6, alpha=0.5)
axes[1].axhline(y=50, color='white', linestyle='--', linewidth=0.6, alpha=0.5)

cbar = fig.colorbar(im, ax=axes[1], pad=0.01)
cbar.set_label('PSD (dB re µV²/Hz)', fontsize=10)

plt.tight_layout()
plt.show()

print('Perhatikan peningkatan warna (daya) pada frekuensi 8-13 Hz (alpha) selama t=10-15 s.')
print('Spectrogram memungkinkan kita melihat KAPAN dan pada FREKUENSI BERAPA kejadian terjadi.')

In [ ]:
# Spectrogram ECG noisy -- garis 50 Hz terlihat sebagai pita horizontal
nperseg_ecg  = int(fs_ecg * 2)    # window 2 detik
noverlap_ecg = int(nperseg_ecg * 0.85)

f_spec_ecg_c, t_spec_ecg_c, Sxx_ecg_c = scipy_signal.spectrogram(
    ecg_clean, fs=fs_ecg,
    nperseg=nperseg_ecg, noverlap=noverlap_ecg,
    window='hann', scaling='density'
)
f_spec_ecg_n, t_spec_ecg_n, Sxx_ecg_n = scipy_signal.spectrogram(
    ecg_noisy, fs=fs_ecg,
    nperseg=nperseg_ecg, noverlap=noverlap_ecg,
    window='hann', scaling='density'
)

vmin = -60
vmax = 10

fig, axes = plt.subplots(2, 1, figsize=(14, 9))
fig.suptitle('Spectrogram ECG: Bersih vs. Terkontaminasi (50 Hz Power Line)', fontsize=12, fontweight='bold')

for ax, Sxx, title in [
    (axes[0], Sxx_ecg_c, 'ECG Bersih'),
    (axes[1], Sxx_ecg_n, 'ECG Terkontaminasi (50 Hz + Baseline Wander + Random Noise)')
]:
    im = ax.pcolormesh(
        t_spec_ecg_c, f_spec_ecg_c,
        10 * np.log10(Sxx + 1e-15),
        shading='gouraud', cmap='viridis',
        vmin=vmin, vmax=vmax
    )
    ax.set_ylabel('Frekuensi (Hz)')
    ax.set_title(title)
    ax.set_ylim([0, 80])
    cbar = fig.colorbar(im, ax=ax, pad=0.01)
    cbar.set_label('dB re mV²/Hz', fontsize=9)

# Tandai 50 Hz di plot kedua
axes[1].axhline(y=50, color='red', linestyle='--', linewidth=1.5, label='50 Hz (power line)')
axes[1].legend(loc='upper right')
axes[1].set_xlabel('Waktu (s)')

plt.tight_layout()
plt.show()

print('Di spectrogram ECG noisy, terlihat:')
print('  - Pita horizontal terang pada 50 Hz --> interferensi jala listrik persisten.')
print('  - Peningkatan daya di frekuensi rendah (< 1 Hz) --> baseline wander.')
print('  - Peningkatan daya merata di semua frekuensi --> random noise (white noise).')

In [ ]:
# Demonstrasi trade-off resolusi waktu vs. frekuensi pada spectrogram EEG
window_configs = [
    (int(fs_eeg * 0.5),  'Window 0.5 s -- Resolusi waktu tinggi, frekuensi rendah'),
    (int(fs_eeg * 2),    'Window 2 s -- Keseimbangan'),
    (int(fs_eeg * 8),    'Window 8 s -- Resolusi frekuensi tinggi, waktu rendah'),
]

fig, axes = plt.subplots(3, 1, figsize=(14, 13))
fig.suptitle('Trade-off Resolusi Waktu vs. Frekuensi (Prinsip Heisenberg)', fontsize=12, fontweight='bold')

for ax, (nperseg_cfg, label) in zip(axes, window_configs):
    noverlap_cfg = int(nperseg_cfg * 0.85)
    f_c, t_c, Sxx_c = scipy_signal.spectrogram(
        eeg_clean, fs=fs_eeg,
        nperseg=nperseg_cfg, noverlap=noverlap_cfg,
        window='hann', scaling='density'
    )
    delta_f_c = f_c[1] - f_c[0]
    delta_t_c = t_c[1] - t_c[0] if len(t_c) > 1 else nperseg_cfg / fs_eeg

    im = ax.pcolormesh(
        t_c, f_c,
        10 * np.log10(Sxx_c + 1e-15),
        shading='gouraud', cmap='inferno'
    )
    ax.set_title(f'{label} | Δt≈{delta_t_c:.2f} s, Δf={delta_f_c:.3f} Hz', fontsize=10)
    ax.set_ylabel('Frekuensi (Hz)')
    ax.set_ylim([0, 50])
    ax.axhline(y=13, color='cyan',  linestyle='--', linewidth=0.8, alpha=0.8, label='Alpha batas atas (13 Hz)')
    ax.axhline(y=8,  color='white', linestyle='--', linewidth=0.8, alpha=0.8, label='Alpha batas bawah (8 Hz)')
    ax.legend(loc='upper right', fontsize=7)
    fig.colorbar(im, ax=ax, pad=0.01).set_label('dB', fontsize=8)

axes[-1].set_xlabel('Waktu (s)')
plt.tight_layout()
plt.show()

print('Pengamatan prinsip ketidakpastian:')
print('  - Window 0.5 s : Alpha burst terlihat jelas sebagai pita sempit di waktu,')
print('    namun batas pita 8-13 Hz sulit dibedakan (resolusi frekuensi kasar).')
print('  - Window 8 s   : Pita alpha (8-13 Hz) terlihat sangat tajam, namun transisi')
print('    awal/akhir burst tidak jelas (resolusi waktu rendah).')
print('  - Window 2 s   : Kompromi -- alpha burst dan pita frekuensinya keduanya terlihat.')

---
## 6. Kesimpulan

### Ringkasan Materi Minggu 3: Analisis Frekuensi

1. **Motivasi analisis frekuensi**: Domain waktu tidak cukup untuk mengidentifikasi dan memisahkan komponen-komponen sinyal biomedis. Domain frekuensi memungkinkan kita melihat *apa* frekuensi yang ada, *seberapa kuat* masing-masing, dan *kapan* komponen tersebut aktif.

2. **DFT dan FFT**: DFT mentransformasi sinyal $N$ sampel ke $N$ bin frekuensi. FFT adalah algoritma efisien $O(N \log N)$ untuk menghitung DFT. Parameter kunci: resolusi frekuensi $\Delta f = f_s / N$ dan frekuensi Nyquist $f_{\max} = f_s / 2$.

3. **Interpretasi spektrum klinis**: Puncak di 50 Hz menandakan interferensi jala listrik; daya rendah di bawah 0.5 Hz menandakan *baseline wander*; kompleks QRS mendominasi pita 5--40 Hz. Spektrum adalah "sidik jari" komposisi sinyal.

4. **Power Spectral Density (PSD)**: PSD menyatakan distribusi daya per satuan frekuensi ($\text{V}^2/\text{Hz}$). Metode Welch mengurangi variansi estimasi dengan merata-ratakan periodogram dari segmen-segmen yang saling overlap.

5. **Spectrogram (STFT)**: Menggabungkan informasi waktu dan frekuensi dalam satu representasi dua dimensi. Berguna untuk sinyal non-stasioner seperti EEG (transisi kondisi otak) dan ECG (perubahan irama jantung).

6. **Trade-off resolusi**: Prinsip ketidakpastian Heisenberg menyatakan bahwa kita tidak dapat memiliki resolusi waktu dan frekuensi yang sama-sama tinggi. Pemilihan panjang window harus disesuaikan dengan kebutuhan analisis.

### Minggu Depan

Dengan pemahaman spektrum frekuensi sinyal dan noise, kita siap mempelajari cara **menghilangkan** komponen yang tidak diinginkan secara presisi: **Filter Digital** (*Digital Filters*). Kita akan membahas FIR, IIR, bandpass, dan notch filter serta penerapannya pada ECG dan EEG.

---
## Referensi

1. **Rangayyan, R.M.** *Biomedical Signal Analysis*, 2nd Edition. Wiley-IEEE Press, 2015. -- Bab 5: Analisis Frekuensi Sinyal Biomedis.

2. **Sörnmo, L. & Laguna, P.** *Bioelectrical Signal Processing in Cardiac and Neurological Applications*. Academic Press / Elsevier, 2005. -- Bab 3: Spectral Analysis.

3. **Oppenheim, A.V. & Schafer, R.W.** *Discrete-Time Signal Processing*, 3rd Edition. Prentice Hall, 2010. -- Bab 8: The Discrete Fourier Transform.

4. **Proakis, J.G. & Manolakis, D.G.** *Digital Signal Processing: Principles, Algorithms, and Applications*, 4th Edition. Pearson, 2007.

5. **SciPy Documentation**: `scipy.signal.welch`, `scipy.signal.periodogram`, `scipy.signal.spectrogram` -- https://docs.scipy.org/doc/scipy/reference/signal.html